# Notebook 6: Controlled SDE Simulation\n\n**Author:** Divyansh Atri\n\n## Overview\n\nMonte Carlo simulation to verify HJB solutions through closed-loop control.\n\n**Topics:**\n1. Euler-Maruyama for controlled SDEs\n2. Feedback control implementation\n3. Empirical cost computation\n4. Comparison with value function\n5. Statistical analysis

In [ ]:
import numpy as np\nimport matplotlib.pyplot as plt\nimport sys\nsys.path.append('..')\nfrom utils import *\n\nplt.style.use('seaborn-v0_8-darkgrid')\nplt.rcParams['figure.figsize'] = (14, 6)\n\nnp.random.seed(42)\nprint('Controlled SDE Simulation - Ready')

## 1. Problem Setup\n\nWe'll simulate the controlled Brownian motion problem and verify that:\n$$V(0, x_0) \approx \mathbb{E}[J(u^*)]$$\n\nwhere $u^*$ is the optimal feedback control from the HJB solution.

In [ ]:
# Setup\nmodel = ControlledBrownianMotion(sigma=0.5)\ncost_fn = QuadraticCost(q=1.0, r=1.0, q_terminal=10.0)\n\nx_min, x_max, nx = -3.0, 3.0, 101\nT, nt = 2.0, 201\n\nprint(f'Model: Controlled Brownian Motion, σ={model.sigma}')\nprint(f'Cost: q={cost_fn.q}, r={cost_fn.r}, q_T={cost_fn.q_terminal}')\nprint(f'Time horizon: T={T}')

## 2. Solve HJB Equation

In [ ]:
# Solve for optimal control policy\nsolver = HJBSolver(x_min, x_max, nx, T, nt, model, cost_fn)\n\nprint('Solving HJB equation...')\nV, u_opt = solver.solve_backward(u_bounds=(-5, 5), verbose=False)\n\nprint(f'HJB solution complete')\nprint(f'V(0, 0) = {V[0, nx//2]:.6f}')

## 3. Define Feedback Control Policy

In [ ]:
# Create policy function from HJB solution\ndef feedback_policy(t, x):\n    '''Optimal control from HJB solution'''\n    return solver.get_policy(t, x)\n\n# Test policy\ntest_x = np.array([0.0, 0.5, 1.0, -0.5])\ntest_t = 0.0\nprint('Testing feedback policy:')\nfor x_val in test_x:\n    u_val = feedback_policy(test_t, x_val)\n    print(f'  u*({test_t}, {x_val:5.2f}) = {u_val:7.4f}')

## 4. Monte Carlo Simulation

In [ ]:
# Simulation parameters\nx0 = 0.0\ndt_sim = 0.01\nn_paths = 1000\n\nprint(f'\nRunning Monte Carlo simulation...')\nprint(f'  Initial state: x0 = {x0}')\nprint(f'  Number of paths: {n_paths}')\nprint(f'  Time step: dt = {dt_sim}')\n\n# Create simulator\nsimulator = ClosedLoopSimulator(model, cost_fn, feedback_policy)\n\n# Run simulation\nresults = simulator.simulate(x0, T, dt_sim, n_paths=n_paths, seed=42)\n\nprint(f'\nSimulation complete!')\nprint(f'  Mean cost: {results["mean_cost"]:.6f}')\nprint(f'  Std cost: {results["std_cost"]:.6f}')\nprint(f'  V(0, {x0}): {V[0, nx//2]:.6f}')\nprint(f'  Difference: {abs(results["mean_cost"] - V[0, nx//2]):.6f}')

## 5. Visualization of Trajectories

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))\n\nt_sim = results['t']\nx_paths = results['x_paths']\nu_paths = results['u_paths']\n\n# Plot sample trajectories\nn_plot = min(50, n_paths)\nfor i in range(n_plot):\n    axes[0,0].plot(t_sim, x_paths[i, :], 'b-', alpha=0.3, linewidth=0.5)\naxes[0,0].plot(t_sim, results['x_mean'], 'r-', linewidth=3, label='Mean')\naxes[0,0].fill_between(t_sim, \n                       results['x_mean'] - results['x_std'],\n                       results['x_mean'] + results['x_std'],\n                       alpha=0.3, color='red', label='±1 std')\naxes[0,0].axhline(0, color='k', linestyle='--', alpha=0.5)\naxes[0,0].set_xlabel('Time $t$')\naxes[0,0].set_ylabel('State $X_t$')\naxes[0,0].set_title(f'State Trajectories ({n_plot} paths shown)')\naxes[0,0].legend()\naxes[0,0].grid(True, alpha=0.3)\n\n# Plot control trajectories\nfor i in range(n_plot):\n    axes[0,1].plot(t_sim, u_paths[i, :], 'g-', alpha=0.3, linewidth=0.5)\naxes[0,1].plot(t_sim, results['u_mean'], 'r-', linewidth=3, label='Mean')\naxes[0,1].axhline(0, color='k', linestyle='--', alpha=0.5)\naxes[0,1].set_xlabel('Time $t$')\naxes[0,1].set_ylabel('Control $u_t$')\naxes[0,1].set_title(f'Control Trajectories ({n_plot} paths shown)')\naxes[0,1].legend()\naxes[0,1].grid(True, alpha=0.3)\n\n# Cost distribution\naxes[1,0].hist(results['costs'], bins=50, density=True, alpha=0.7, edgecolor='black')\naxes[1,0].axvline(results['mean_cost'], color='r', linestyle='--', linewidth=2, label=f'Mean: {results["mean_cost"]:.3f}')\naxes[1,0].axvline(V[0, nx//2], color='b', linestyle='--', linewidth=2, label=f'V(0,0): {V[0, nx//2]:.3f}')\naxes[1,0].set_xlabel('Cost $J$')\naxes[1,0].set_ylabel('Density')\naxes[1,0].set_title('Empirical Cost Distribution')\naxes[1,0].legend()\naxes[1,0].grid(True, alpha=0.3)\n\n# Terminal state distribution\naxes[1,1].hist(x_paths[:, -1], bins=50, density=True, alpha=0.7, edgecolor='black')\naxes[1,1].axvline(np.mean(x_paths[:, -1]), color='r', linestyle='--', linewidth=2, label=f'Mean: {np.mean(x_paths[:, -1]):.3f}')\naxes[1,1].set_xlabel('Terminal state $X_T$')\naxes[1,1].set_ylabel('Density')\naxes[1,1].set_title('Terminal State Distribution')\naxes[1,1].legend()\naxes[1,1].grid(True, alpha=0.3)\n\nplt.tight_layout()\nplt.savefig('../plots/simulation_results.png', dpi=150, bbox_inches='tight')\nplt.show()

## 6. Verification Analysis

In [ ]:
# Compute confidence interval\nfrom scipy import stats\n\nmean_cost = results['mean_cost']\nstd_cost = results['std_cost']\nn = n_paths\n\n# 95% confidence interval\nci_95 = stats.t.interval(0.95, n-1, loc=mean_cost, scale=std_cost/np.sqrt(n))\n\nprint('\nVerification Results:')\nprint('=' * 50)\nprint(f'Value function V(0, 0): {V[0, nx//2]:.6f}')\nprint(f'Empirical mean cost:    {mean_cost:.6f}')\nprint(f'95% CI:                 [{ci_95[0]:.6f}, {ci_95[1]:.6f}]')\nprint(f'Difference:             {abs(mean_cost - V[0, nx//2]):.6f}')\nprint(f'Relative error:         {abs(mean_cost - V[0, nx//2])/V[0, nx//2]*100:.3f}%')\n\nif ci_95[0] <= V[0, nx//2] <= ci_95[1]:\n    print('✓ Value function is within 95% CI - VERIFIED!')\nelse:\n    print('✗ Value function outside 95% CI - check implementation')

## 7. Comparison with Different Initial States

In [ ]:
# Test multiple initial states\nx0_values = np.linspace(-2, 2, 9)\nempirical_costs = []\ntheoretical_costs = []\n\nprint('\nTesting different initial states...')\nfor x0_test in x0_values:\n    # Simulate\n    res = simulator.simulate(x0_test, T, dt_sim, n_paths=500, seed=None)\n    empirical_costs.append(res['mean_cost'])\n    \n    # Get theoretical value\n    x_idx = np.argmin(np.abs(solver.x - x0_test))\n    theoretical_costs.append(V[0, x_idx])\n    \n    print(f'  x0={x0_test:5.2f}: Empirical={res["mean_cost"]:7.4f}, Theory={V[0, x_idx]:7.4f}')\n\n# Plot comparison\nplt.figure(figsize=(10, 6))\nplt.plot(x0_values, theoretical_costs, 'b-o', linewidth=2, markersize=8, label='HJB Solution $V(0,x)$')\nplt.plot(x0_values, empirical_costs, 'r--s', linewidth=2, markersize=8, label='Empirical Cost')\nplt.xlabel('Initial state $x_0$')\nplt.ylabel('Cost')\nplt.title('Verification Across Initial States')\nplt.legend()\nplt.grid(True, alpha=0.3)\nplt.tight_layout()\nplt.savefig('../plots/verification_multiple_x0.png', dpi=150, bbox_inches='tight')\nplt.show()

## Summary\n\n**Key Results:**\n1. Monte Carlo simulation confirms HJB solution\n2. Empirical costs match theoretical value function\n3. Feedback control successfully stabilizes the system\n4. Statistical verification across multiple initial states\n\nThis provides strong empirical evidence for the correctness of our numerical methods.\n\n**Next:** Summary and discussion of limitations.